# Установка gensim на воркер

Установка conda:
* `https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh`
* `chmod +x Anaconda3-2019.10-Linux-x86_64.sh`
* `./Anaconda3-2019.10-Linux-x86_64.sh`  (если случайно запустить с sudo, то потом [поменять владельца](https://github.com/conda/conda/issues/7267#issuecomment-498651734))
* отключить авто-init: `conda config --set auto_activate_base false`

Gensim:
* `conda install gensim`

Jupyter:
* `conda install ipykernel`
* запуск сервера: `jupyter notebook --no-browser --port=8888 --ip='::'`
* выбрать кернел



In [1]:
import gensim

# Качаем данные с википедии

In [2]:
# https://dumps.wikimedia.org/ruwiki/20191220/
!wget https://dumps.wikimedia.org/ruwiki/20191220/ruwiki-20191220-pages-articles-multistream.xml.bz2

--2019-12-25 12:51:41--  https://dumps.wikimedia.org/ruwiki/20191220/ruwiki-20191220-pages-articles-multistream.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|2620:0:861:1:208:80:154:7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3992310344 (3,7G) [application/octet-stream]
Saving to: `ruwiki-20191220-pages-articles-multistream.xml.bz2'

100%[====================================>] 3 992 310 344 4,88M/s   in 12m 50s 

2019-12-25 13:04:33 (4,94 MB/s) - `ruwiki-20191220-pages-articles-multistream.xml.bz2' saved [3992310344/3992310344]



In [1]:
from datetime import datetime

log_filename = 'DL_chapter_7_word2vec_in_memory.log'

def log(msg):
    msg = '[{}] {}'.format(datetime.now(), msg)
    print(msg)
    with open(log_filename, 'a') as fout:
        fout.write(msg + '\n')

# Загрузка и предобработка данных

*Токенизация*: разбиение потока символов на слова (+ очистка от лишних символов)

*Лемматизация*: приведение слов к начальным формам. Мы не будем делать, т.к. во-первых хотим посмотреть на всякие эффекты, а во-вторых, у нас и так достаточно данных

In [2]:
from gensim.corpora.wikicorpus import WikiCorpus

wiki = WikiCorpus('ruwiki-20191220-pages-articles-multistream.xml.bz2', dictionary=False)

Данные не читаются с диска в память, а благодаря dictionary=False не строится и словарь

*Биграмма* — это пара слов `(w1,w2)`, которые вместе встречаются аномально часто, то есть совместная вероятность их появления `p(w1,w2)`
в тексте существенно выше, чем `p(w1)p(w2)`

Есть разные алгоритмы выделения биграмм, но в Gensim уже есть реализация

In [3]:
def get_texts_step(step=100):
    # get every [step]th text
    for i, text in enumerate(wiki.get_texts()):
        if i % step == 0:
            yield text

log("Started loading texts")
all_texts = list(wiki.get_texts())
log("Started loading texts")


def get_texts():
    # get texts from memory
    for text in all_texts:
        yield text

[2019-12-25 16:12:57.572576] Started loading texts
[2019-12-25 16:41:07.762148] Started loading texts


In [4]:
from gensim.models.phrases import Phrases, Phraser

# генератор биграм

log("Started building bigrams")
bigram = Phrases(get_texts())
bigram_transformer = Phraser(bigram)
log("Finished building bigrams")


def text_generator_bigram():
    for text in get_texts():
        yield bigram_transformer[ [ word for word in text ] ]   # .decode('utf-8')

[2019-12-25 16:57:19.300192] Started building bigrams
[2019-12-25 17:31:09.662020] Finished building bigrams


In [5]:
# генератор триграм

log("Started building trigrams")
trigram = Phrases(text_generator_bigram())
trigram_transformer = Phraser(trigram)
log("Finished building trigrams")

def text_generator_trigram():
    for text in get_texts():
        yield trigram_transformer[ bigram_transformer[ [ word for word in text ] ] ]   # .decode('utf-8')

[2019-12-25 17:31:09.669413] Started building trigrams
[2019-12-25 18:29:54.008252] Finished building trigrams


# Обучаем word2vec на последовательности триграмм

## Обучаю модель size=100, epochs=25

In [6]:
log("Started counting examples")
total_examples = 0
for trigram in text_generator_trigram():
    total_examples += 1
log("Finished counting examples, result: {}".format(total_examples))

[2019-12-25 18:29:54.014886] Started counting examples
[2019-12-25 19:26:06.959298] Finished counting examples, result: 1331252


In [7]:
from gensim.models.word2vec import Word2Vec

model_100 = Word2Vec(size=100, window=7, min_count=10, workers=32)

log("Started building vocab")
model_100.build_vocab( text_generator_trigram() )
log("Finished building vocab")

[2019-12-25 19:26:06.966888] Started building vocab
[2019-12-25 20:37:31.822850] Finished building vocab


In [8]:
log("Started training")
model_100.train( text_generator_trigram(), total_examples=total_examples, epochs=25 )
log("Finished training")

[2019-12-25 20:37:31.828769] Started training
[2019-12-25 21:39:03.319412] Finished training


In [9]:
model_100.save('DL_chapter_7_word2vec_model_size_100_percent_100_epochs_25')

* size -- размерность вектора представления
* window -- размер окна локального контекста
* min_count -- минимальная частота встречаемости, чтобы мы рассматривали слово
* workers -- параллелизация

## Обучаю модель size=500, epochs=25

In [10]:
model_500 = Word2Vec(size=500, window=7, min_count=10, workers=32)

log("Started building vocab")
model_500.build_vocab( text_generator_trigram() )
log("Finished building vocab")

[2019-12-25 21:39:12.828759] Started building vocab
[2019-12-25 22:48:20.032481] Finished building vocab


In [11]:
log("Started training")
model_500.train( text_generator_trigram(), total_examples=total_examples, epochs=25 )
log("Finished training")

[2019-12-25 22:48:20.039712] Started training
[2019-12-25 23:50:28.923686] Finished training


In [12]:
model_500.save('DL_chapter_7_word2vec_model_size_500_percent_100_epochs_25')

## Обучаю модель size=500, epochs=200

In [13]:
model_500_200 = Word2Vec(size=500, window=7, min_count=10, workers=32)

log("Started building vocab")
model_500_200.build_vocab( text_generator_trigram() )
log("Finished building vocab")

[2019-12-25 23:50:49.341133] Started building vocab
[2019-12-26 00:59:23.792330] Finished building vocab


In [14]:
log("Started training")
model_500_200.train( text_generator_trigram(), total_examples=total_examples, epochs=200 )
log("Finished training")

[2019-12-26 00:59:23.830482] Started training
[2019-12-26 02:01:54.071193] Finished training


In [15]:
model_500_200.save('DL_chapter_7_word2vec_model_size_500_percent_100_epochs_200')

In [16]:
model_500_200.wv.most_similar(
  ['актёр', 'женщина'],
  ['мужчина']
)

[('актриса', 0.6711472868919373),
 ('ведущая_актриса', 0.5662055611610413),
 ('актер', 0.5578432679176331),
 ('сценарист_режиссёр', 0.5533250570297241),
 ('киноактриса', 0.5437122583389282),
 ('благодаря_ролям', 0.5435642600059509),
 ('снимавшаяся', 0.5400126576423645),
 ('балерина', 0.5322019457817078),
 ('театра_кино', 0.527100682258606),
 ('танцовщица', 0.5250095725059509)]

In [17]:
model_500_200.wv.most_similar(
  ['математик', 'женщина'],
  ['мужчина']
)

[('доктор_философии', 0.6313427686691284),
 ('итальянский_физик', 0.6195719242095947),
 ('учёная', 0.6180987358093262),
 ('немецкий_философ', 0.6100102663040161),
 ('немецкий_математик', 0.6081817746162415),
 ('филолог', 0.6068433523178101),
 ('физиолог', 0.6051205396652222),
 ('лингвист', 0.6050077676773071),
 ('доктор_филологических_наук', 0.6020523309707642),
 ('американский_математик', 0.5991174578666687)]

In [19]:
model_500_200.wv.most_similar(
  ['математики', 'мальчик'],
  ['математик']
)

[('малыша', 0.5166980624198914),
 ('стьюи', 0.5018874406814575),
 ('картман', 0.5018465518951416),
 ('мальчика', 0.5016962885856628),
 ('гомер', 0.49474072456359863),
 ('перед_телевизором', 0.4932996928691864),
 ('мальчиком', 0.4908002018928528),
 ('куклу', 0.4742394685745239),
 ('гулять', 0.4728545546531677),
 ('каждое_утро', 0.4712660312652588)]

In [22]:
model_500_200.wv.most_similar(
  ['львица', 'кошка'],
  ['лев']
)

[('маленькая_девочка', 0.7352694869041443),
 ('блондинка', 0.7277578115463257),
 ('птичка', 0.7274823188781738),
 ('собачка', 0.7228641510009766),
 ('лягушка', 0.72053062915802),
 ('красивая_женщина', 0.7199786305427551),
 ('молодая_девушка', 0.7133882641792297),
 ('красивая', 0.7090722322463989),
 ('волшебница', 0.7085510492324829),
 ('мёртвая', 0.7060226202011108)]

In [23]:
model_500_200.wv.most_similar(
  ['машина_времени']
)

[('агата_кристи', 0.84107905626297),
 ('чайф', 0.8366405963897705),
 ('наутилус_помпилиус', 0.8267172574996948),
 ('наив', 0.8230922222137451),
 ('король_шут', 0.821985125541687),
 ('мумий_тролль', 0.821345865726471),
 ('animal_джаz', 0.8113285303115845),
 ('весёлые_ребята', 0.8061252236366272),
 ('сплин', 0.8030151724815369),
 ('любэ', 0.8004089593887329)]

In [26]:
model_500_200.wv.most_similar(
  ['машинное_обучение']
)

[('искусственные_нейронные_сети', 0.8848088383674622),
 ('тестирование_программного_обеспечения', 0.8839117884635925),
 ('параллельные_вычисления', 0.8781010508537292),
 ('обработка_естественного_языка', 0.8728587627410889),
 ('эконометрика', 0.8694239854812622),
 ('биоинформатика', 0.8652983903884888),
 ('теория_алгоритмов', 0.8634968996047974),
 ('теория_принятия_решений', 0.852242112159729),
 ('статьи_примерами_кода', 0.8514398336410522),
 ('объектно_ориентированное', 0.8513944745063782)]

In [27]:
model_500_200.wv.most_similar(
  ['яндекс']
)

[('mail_ru', 0.8292312622070312),
 ('рамблер', 0.8027689456939697),
 ('яндекса', 0.7869933843612671),
 ('сервис', 0.7727541923522949),
 ('google', 0.7727051973342896),
 ('мобильное_приложение', 0.7631986141204834),
 ('сервиса', 0.7597266435623169),
 ('сети_интернет', 0.7447307705879211),
 ('cnews', 0.7446727752685547),
 ('icq', 0.7443001866340637)]

In [28]:
model_100.wv.most_similar(
  ['яндекс']
)

[('mail_ru', 0.8767240643501282),
 ('яндекса', 0.8632349967956543),
 ('google', 0.8461947441101074),
 ('сервис', 0.8348525762557983),
 ('веб', 0.8075309991836548),
 ('сети_интернет', 0.8010289072990417),
 ('сервиса', 0.7911575436592102),
 ('онлайн', 0.7875292897224426),
 ('мобильное_приложение', 0.7864222526550293),
 ('мобильного_приложения', 0.7859923243522644)]

In [29]:
model_500.wv.most_similar(
  ['яндекс']
)

[('mail_ru', 0.8320544958114624),
 ('яндекса', 0.8221307992935181),
 ('google', 0.7737005949020386),
 ('сервис', 0.7698644399642944),
 ('сервиса', 0.7655315399169922),
 ('хостинга', 0.7606269121170044),
 ('хостинг', 0.7500610947608948),
 ('мобильное_приложение', 0.7489928603172302),
 ('icq', 0.7475957870483398),
 ('открытый_доступ', 0.7470940351486206)]

In [30]:
model_500_200.wv.most_similar(
  ['много']
)

[('немало', 0.7261830568313599),
 ('большое_количество', 0.6664397716522217),
 ('множество', 0.6199277639389038),
 ('огромное_количество', 0.6102041006088257),
 ('большое_число', 0.5960496664047241),
 ('много_хороших', 0.57206130027771),
 ('значительное_количество', 0.5666031837463379),
 ('несколько', 0.5474859476089478),
 ('целый_ряд', 0.541946291923523),
 ('много_интересных', 0.5414936542510986)]

In [31]:
model_500_200.wv.most_similar(
  ['френдзона']
)

[('сексизм', 0.5492262840270996),
 ('зоофилию', 0.5465393662452698),
 ('бюрократизм', 0.5432154536247253),
 ('японский_национализм', 0.5407375693321228),
 ('шовинизм', 0.5390805006027222),
 ('вуайеризм', 0.5389503240585327),
 ('манию', 0.5375597476959229),
 ('субкультуру', 0.5363483428955078),
 ('садизм', 0.5327458381652832),
 ('маскулизм', 0.5308481454849243)]

In [33]:
model_500_200.wv.most_similar(
  ['аркадий_волож']
)

[('альфа_групп', 0.7112696170806885),
 ('retail_group', 0.7049852609634399),
 ('виктор_вексельберг', 0.7047931551933289),
 ('газпромбанк', 0.704300045967102),
 ('алишер_усманов', 0.7005768418312073),
 ('московский_кредитный_банк', 0.7000415325164795),
 ('роснано', 0.6892472505569458),
 ('владимир_потанин', 0.6842904090881348),
 ('сибирский_деловой_союз', 0.6832914352416992),
 ('интеррос', 0.6813831925392151)]

In [34]:
model_500_200.wv.most_similar(
  ['илья_сегалович']
)

[('аркадий_волож', 0.6262919902801514),
 ('виталий_геннадьевич', 0.6228498220443726),
 ('владимир_потанин', 0.6207123398780823),
 ('клишин', 0.6134703159332275),
 ('александр_несис', 0.6128029227256775),
 ('эдуард_сагалаев', 0.6087579727172852),
 ('аналитическая_программа_рунетология', 0.6080464124679565),
 ('иосиф_бакштейн', 0.6072640419006348),
 ('петр_авен', 0.6071577668190002),
 ('ашурков', 0.6059235334396362)]

In [39]:
model_500_200.wv.most_similar(
   ['пять'],
   ['четыре'] 
)

[('более_ста', 0.34239163994789124),
 ('девять', 0.33672934770584106),
 ('ряде_других', 0.32871773838996887),
 ('семь', 0.3239850699901581),
 ('шесть', 0.32239317893981934),
 ('обеих', 0.32094478607177734),
 ('десять_тысяч', 0.3130486309528351),
 ('десять', 0.304669052362442),
 ('тринадцать', 0.30356496572494507),
 ('ряде', 0.30335530638694763)]

In [41]:
model_500_200.wv.most_similar(
   ['ряде_других'],
)

[('ряде', 0.738898754119873),
 ('целом_ряде', 0.6918152570724487),
 ('во_множестве', 0.6814412474632263),
 ('десятках', 0.6229119300842285),
 ('многих_других', 0.6152180433273315),
 ('большинстве', 0.6084815263748169),
 ('нескольких_десятках', 0.5894585251808167),
 ('ряде_европейских', 0.5705533623695374),
 ('зарубежных', 0.546991229057312),
 ('европейских_американских', 0.5431177020072937)]

In [42]:
model_500_200.wv.most_similar(
   ['ноль'],
)

[('плюс_минус', 0.6705114841461182),
 ('умножается', 0.6577564477920532),
 ('каждом_шаге', 0.6283431053161621),
 ('кубик', 0.6213582754135132),
 ('умножение', 0.6134402751922607),
 ('слагаемых', 0.6132329702377319),
 ('выбираем', 0.6124820113182068),
 ('нулевой', 0.6115670204162598),
 ('предположим_что', 0.6079829335212708),
 ('выберем', 0.6062613725662231)]

In [43]:
model_500_200.wv.most_similar(
   ['нуль'],
)

[('ненулевой', 0.86075758934021),
 ('предположим_что', 0.8460514545440674),
 ('тензор', 0.8413795232772827),
 ('каждой_точке', 0.8396062850952148),
 ('преобразование_фурье', 0.8393461108207703),
 ('энтропии', 0.8391460180282593),
 ('общем_случае', 0.8386926054954529),
 ('xor', 0.8364247679710388),
 ('двоичной', 0.835400402545929),
 ('гамильтониан', 0.8342263102531433)]

In [44]:
model_500_200.wv.most_similar(
   ['xor'],
)

[('const', 0.8707807064056396),
 ('умножение', 0.8688753843307495),
 ('конъюнкция', 0.8675186634063721),
 ('двоичной', 0.8656684756278992),
 ('открытого_текста', 0.8652310967445374),
 ('инициализация', 0.8619481325149536),
 ('шифротекста', 0.8614895939826965),
 ('операндов', 0.8508354425430298),
 ('преобразование_фурье', 0.8500239253044128),
 ('null', 0.8493994474411011)]

In [45]:
model_500_200.wv.most_similar(
    ['король', 'женщина'],
    ['мужчина']
)

[('королева', 0.5937466621398926),
 ('португалия_афонсу_африканец_король', 0.5323777794837952),
 ('императрица', 0.5213098526000977),
 ('принцесса', 0.5115206837654114),
 ('правительница', 0.5036981701850891),
 ('король_португалии', 0.5027424097061157),
 ('филипп_iv', 0.5024620294570923),
 ('дож_византийская_империя', 0.49978893995285034),
 ('генрих_ii', 0.49829164147377014),
 ('империя', 0.4957278370857239)]

In [46]:
model_500_200.wv.most_similar(
    ['император', 'женщина'],
    ['мужчина']
)

[('императрица', 0.659258246421814),
 ('императора', 0.6167657971382141),
 ('вдовствующая_императрица', 0.6006978154182434),
 ('императором', 0.5763481259346008),
 ('византия', 0.5729753971099854),
 ('цыси', 0.5658361911773682),
 ('правительница', 0.5556811094284058),
 ('царица', 0.5528994798660278),
 ('византийская_империя', 0.5437092781066895),
 ('период_трех_государств', 0.5427613854408264)]

In [ ]:
model_500_200.wv.most_similar(
    ['король', 'женщина'],
    ['мужчина']
)

In [51]:
type(trigram_transformer.phrasegrams)

dict